In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys

!git clone https://github.com/kymatio/kymatio.git
sys.path.append('/content/kymatio')
# !pip install -r kymatio/requirements.txt
# !python kymatio/setup.py install
# #!pip uninstall kymatio
# sys.path.append('/content/gdrive/MyDrive/scatter_signal_typing')
!pip install -e /content/kymatio

Mounted at /content/gdrive
Cloning into 'kymatio'...
remote: Enumerating objects: 5108, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 5108 (delta 212), reused 320 (delta 197), pack-reused 4740
Receiving objects: 100% (5108/5108), 2.17 MiB | 7.46 MiB/s, done.
Resolving deltas: 100% (3266/3266), done.
Obtaining file:///content/kymatio
  Running setup.py develop for kymatio


In [2]:
sys.path.append('/content/gdrive/MyDrive/grbas_capev')

import numpy as np
import os
from scipy.io import wavfile
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
import math
import pickle
from utils import *

from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor

import torch
from kymatio.torch import Scattering1D

In [3]:
T = 2**16
file_path = '/content/gdrive/MyDrive/python_notebooks/grbas_capev/'
file_names = pd.read_csv(file_path + 'file_names.csv', header=None)
data = np.zeros((file_names.shape[1],T))
capev = pd.read_csv(file_path + 'capev_mean_ratings.csv', index_col = 0, header=0)
capev = capev.to_numpy()
labels = capev # 0: Severity, 1: Roughness, 2: Breathiness, 3: Strain, 
                    # 4: Pitch , 5: Loudness.

# Loading ratings matrices for category
severity = pd.read_csv(file_path + 'CAPE-V_Severity.csv', index_col = 0, header=0)
severity = severity.iloc[:,2::].to_numpy()

roughness = pd.read_csv(file_path + 'CAPE-V_Roughness.csv', index_col = 0, header=0)
roughness = roughness.iloc[:,2::].to_numpy()

breathiness = pd.read_csv(file_path + 'CAPE-V_Breathiness.csv', index_col = 0, header=0)
breathiness = breathiness.iloc[:,2::].to_numpy()

strain = pd.read_csv(file_path + 'CAPE-V_Strain.csv', index_col = 0, header=0)
strain = strain.iloc[:,2::].to_numpy()

pitch = pd.read_csv(file_path + 'CAPE-V_Pitch.csv', index_col = 0, header=0)
pitch = pitch.iloc[:,2::].to_numpy()

loudness = pd.read_csv(file_path + 'CAPE-V_Loudness.csv', index_col = 0, header=0)
loudness = loudness.iloc[:,2::].to_numpy()


for i in range(file_names.shape[1]):
  # print(file_names.iloc[0,i])
  _, x = wavfile.read(file_path +'sus_vocal_audio_files/'+ file_names.iloc[0,i])
  x = x/np.max(np.abs(x))
  L = np.max(x.shape)
  if L > T:
    data[i,:] = x[int(L // 2 - T // 2):int(L // 2 + T // 2)]
  else:
    start = (T - len(x)) // 2
    data[i,start:start + len(x)] = x

data = torch.tensor(data)
data = data.contiguous()
data = data.type(dtype=torch.float32)

In [4]:
def runHyperparametersOpt(data, order_coeffs, type_feats):
  # Parameters and hyperparameters array
  J = np.array([i for i in range(3,14)])
  G = np.arange(0, 1.1, 0.1)
  min_rmse_results = dict()


  mse = np.zeros((len(J),len(G)))
  std_mse = np.zeros((len(J),len(G)))
  use_cuda = torch.cuda.is_available()
  if use_cuda:
    print('Using cuda...')
  else:
    print('Not using cuda')

  min_mse = math.inf
  for j in range(len(J)):
      # print(J[j])
      for g in range(len(G)):
        X = get_feature_vector(data, J[j], G[g], order_coeffs, type_feats)
          
        # Partition data into hyperparameters determination and validation sets.
        sets = model_selection.train_test_split(X, labels,
                        severity, roughness, breathiness, strain, pitch, loudness,
                                                    test_size = 0.2, shuffle=True,
                                                    random_state = 2)

        X_val, X_hp, y_val, y_hp = sets[0:4]

        X_hp = np.nan_to_num(X_hp)
        X_val = np.nan_to_num(X_val)
        results = hp_fun(X_hp, y_hp, 5)

        (mse[j,g],std_mse[j,g]) = results['overall_rmse']

        if mse[j,g] < min_mse:
          min_mse = mse[j,g]
          min_rmse_results['min_rmse'] = (mse[j,g],std_mse[j,g])
          min_rmse_results['Jmin'] = J[j]
          min_rmse_results['Gmin'] = G[g]
          min_rmse_results['X_val_min'] = X_val
          min_rmse_results['y_val_min'] = y_val
          min_rmse_results['traits'] = sets[4::2]
          
          # sev,rou,bre,srt,pit,lou = sets[4::2]


  print('J = {}, g = {}'.format(min_rmse_results['Jmin'] , min_rmse_results['Gmin']))
  print(min_rmse_results['min_rmse'][0])
  print(min_rmse_results['min_rmse'][1])
  return min_rmse_results

In [5]:
st_coeffs = [['order1'],['order2'],['order1', 'order2']]
type_feats = [['tv'],['mean','sd'],['tv','mean','sd']]

# st_coeffs = [['order0', 'order2']]
# type_feats = [['tv','mean']]

orders_dict = dict()
list_of_results = list()
min_mse = np.zeros((len(st_coeffs),len(type_feats)))

for i, orders in enumerate(st_coeffs):
  arch_dict = dict()
  for j, tf in enumerate(type_feats):
    print(orders+tf)
    min_rmse_results = runHyperparametersOpt(data, orders, tf)
    list_of_results.append(min_rmse_results)
    min_mse[i,j], std_min_mse = min_rmse_results['min_rmse']
    arch_dict['+'.join(tf)]  = [min_mse[i,j], std_min_mse]
  orders_dict['+'.join(orders)] = arch_dict



['order1', 'tv']
Using cuda...
J = 6, g = 0.1
19.45146784449475
6.580789353069332
['order1', 'mean', 'sd']
Using cuda...
J = 5, g = 0.6000000000000001
22.534542676240612
5.3058938307354095
['order1', 'tv', 'mean', 'sd']
Using cuda...
J = 7, g = 0.5
19.558998198307087
5.2899544172562
['order2', 'tv']
Using cuda...
J = 6, g = 0.0
19.733968691340447
6.965975014556738
['order2', 'mean', 'sd']
Using cuda...
J = 10, g = 0.30000000000000004
21.48206002754713
5.55629765973348
['order2', 'tv', 'mean', 'sd']
Using cuda...
J = 6, g = 0.0
19.538221799267077
5.4688650453435494
['order1', 'order2', 'tv']
Using cuda...
J = 6, g = 0.0
19.58209469661281
6.4301564957082125
['order1', 'order2', 'mean', 'sd']
Using cuda...
J = 11, g = 0.2
20.611005345607158
4.324686843603536
['order1', 'order2', 'tv', 'mean', 'sd']
Using cuda...
J = 7, g = 0.30000000000000004
19.281890444337147
4.786694701817686


In [6]:
dic2df(orders_dict)

tv    mean+sd  tv+mean+sd
order1        0  19.451468  22.534543   19.558998
              1   6.580789   5.305894    5.289954
order2        0  19.733969  21.482060   19.538222
              1   6.965975   5.556298    5.468865
order1+order2 0  19.582095  20.611005   19.281890
              1   6.430156   4.324687    4.786695

In [7]:
ind = np.argmin(min_mse, axis=None)
print(ind)
print(min_mse[np.unravel_index(ind, min_mse.shape)])

# Save dictionary in disc to use later in the validation.
with open(file_path + 'results_hp.pkl', 'wb') as f:
    pickle.dump(list_of_results[ind], f)

8
19.281890444337147
